In [1]:
from urllib import request
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
genres=['concert', 'festival', 'spectacle-et-comedie-musicale', 'humour-et-one-wo-man-show', 'sport', 'parc', 'exposition-et-musee', 'theatre', 'cirque', 'salon', 'danse', 'classique-et-opera', 'loisirs-et-tourisme', 'soiree']

In [15]:
def new_str(string):
    return string.replace('[','').replace(']','').replace("'",'').replace("'",'').replace(',','-')

In [16]:
for genre in genres:
    df=pd.read_csv("events/"+genre+".csv")
    for i in range(len(df)):
        df['Artiste'][i]= new_str(df['Artiste'][i])
    df.to_csv("events/"+genre+".csv",index=False)

c:\Users\liaos\anaconda3\envs\webscrapping\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [12]:
genre='festival'
url="https://www.ticketmaster.fr/fr/"+genre+"/p/1"
request_text = request.urlopen(url).read()
htmlpage= BeautifulSoup(request_text, "html")

In [13]:
print(htmlpage.find("title").text)

Billetterie Concert | Billet & Réservation Place Concerts | Ticketmaster


In [8]:
genres=['concert', 'festival', 'spectacle-et-comedie-musicale', 'humour-et-one-wo-man-show', 'sport', 'parc', 'exposition-et-musee', 'theatre', 'cirque', 'salon', 'danse', 'classique-et-opera', 'loisirs-et-tourisme', 'soiree']
events=[]
for genre in genres:
    url="https://www.ticketmaster.fr/fr/"+genre+"/p/1"
    request_text = request.urlopen(url).read()
    htmlpage= BeautifulSoup(request_text, "html")
    if (htmlpage.find('a',{'name':'lienDernierePage'}) == None):
        lastpage=1
    else:
        lastpage = int(htmlpage.find('a',{'name':'lienDernierePage'}).text)
    for i in range(1,lastpage+1):
        url="https://www.ticketmaster.fr/fr/"+genre+"/p/"+str(i)
        request_text = request.urlopen(url).read()
        htmlpage= BeautifulSoup(request_text, "html")
        for item in htmlpage.findAll('article',{'class':'bloc-result bloc-billet-fr'}):
            event = {}
            priceinfo=item.find('div',{'class':'bloc-result-ticket-info'}).find('strong')
            if (priceinfo != None):
                place = item.find('p',{'class':'bloc-result-location'}).text.lstrip('\n').split(' - ')
                event['Date']=item.find('p',{'class':'bloc-result-when'}).text.replace('é','e')
                event['Lieu']=place[0]
                event['Ville']=place[len(place)-1]
                event['Type']=item.find('p',{'class':'bloc-result-type'}).text.replace('\n','')
                event['Prix']=priceinfo.text.lstrip('\n').replace(' €','')
                event['Artiste']=[]
                for artistes in item.findAll('p',{'class':'bloc-result-artist'}):               
                    for artiste in artistes.text.split('\n'):
                        if artiste != ',' and artiste != 'Artiste :' and artiste != '':
                            event['Artiste'].append(artiste.lstrip('Artiste : '))
                event['Image']=item.find('img')['src']
                events.append(event)
df = pd.DataFrame(events)
df

,Date,Lieu,Ville,Type,Prix,Artiste,Image
0,28 avr. 2023,ACCOR ARENA,PARIS,CONCERT - VARIETE INTERNATIONALE,39.00,[IBIZA EXPERIENCE],https://static.ticketmaster.fr/static/images/v...
1,1 au 3 juin 2023,ACCOR ARENA,PARIS,CONCERT - VARIETE ET CHANSON FRANCAISE,35.00,[-M-],https://static.ticketmaster.fr/static/images/v...
2,2 dec. 2023,PARIS LA DEFENSE ARENA,NANTERRE,CONCERT - POP ROCK FOLK,39.00,[STROMAE],https://static.ticketmaster.fr/static/images/v...
3,18 au 19 juin 2023,ZENITH PARIS,PARIS,CONCERT - POP ROCK FOLK,60.50,"[THE BLACK KEYS, SPOON]",https://static.ticketmaster.fr/static/images/v...
4,24 au 25 juin 2023,STADE PIERRE MAUROY,VILLENEUVE D ASCQ,CONCERT - RAP HIP-HOP SLAM,40.00,[SOPRANO],https://static.ticketmaster.fr/static/images/v...
...,...,...,...,...,...,...,...
10618,10 juin au 23 sept. 2023,JARDINS DU CHATEAU DE VERSAILLES,VERSAILLES,LOISIRS ET TOURISME - VISITE ET EXCURSION,29.70,[LES GRANDES EAUX NOCTURNES],https://static.ticketmaster.fr/static/images/v...
10619,31 dec. 2022,PALAIS DE L'EPI D'OR,VILLEJUIF,SOIREE - REVEILLON,25.00,[REVEILLON 2023],https://static.ticketmaster.fr/static/images/v...
10620,31 dec. 2022,New Morning,PARIS,SOIREE - REVEILLON,55.00,[LE GRAND REVEILLON DU NOUVEL AN],https://static.ticketmaster.fr/static/images/v...
10621,6 fevr. 2023,LES FOLIES BERGERE,PARIS,SOIREE - SOIREE MUSIQUE,20.00,"[CORPS A COEURS, LOUANE, BERENGERE KRIEF, JOYC...",https://static.ticketmaster.fr/static/images/v...


In [9]:
df.to_csv('events_1.csv',index=False)

In [4]:
events=[]
url='https://www.ticketmaster.fr/fr/concert/p/1'
request_text = request.urlopen(url).read()
htmlpage= BeautifulSoup(request_text, "html")
for item in htmlpage.findAll('article',{'class':'bloc-result bloc-billet-fr'}):
    event = {}
    priceinfo=item.find('div',{'class':'bloc-result-ticket-info'}).find('strong')
    if (priceinfo != None):
        place = item.find('p',{'class':'bloc-result-location'}).text.lstrip('\n').split(' - ')
        event['Date']=item.find('p',{'class':'bloc-result-when'}).text.replace('é','e')
        event['Lieu']=place[0]
        event['Ville']=place[len(place)-1]
        event['Type']=item.find('p',{'class':'bloc-result-type'}).text.replace('\n','')
        event['Prix']=priceinfo.text.lstrip('\n').replace(' €','')
        event['Artiste']=[]
        for artistes in item.findAll('p',{'class':'bloc-result-artist'}):               
            for artiste in artistes.text.split('\n'):
                if artiste != ',' and artiste != 'Artiste :' and artiste != '':
                    event['Artiste'].append(artiste.lstrip('Artiste : '))
        event['Image']=item.find('img')['src']
        events.append(event)
events

[{'Date': '28 avr. 2023',
  'Lieu': 'ACCOR ARENA',
  'Ville': 'PARIS',
  'Type': 'CONCERT - VARIETE INTERNATIONALE',
  'Prix': '39.00',
  'Artiste': ['IBIZA EXPERIENCE'],
  'img': 'https://static.ticketmaster.fr/static/images/vignettes/n_ibiza-experience.gif'},
 {'Date': '1 au 3 juin 2023',
  'Lieu': 'ACCOR ARENA',
  'Ville': 'PARIS',
  'Type': 'CONCERT - VARIETE ET CHANSON FRANCAISE',
  'Prix': '35.00',
  'Artiste': ['-M-'],
  'img': 'https://static.ticketmaster.fr/static/images/vignettes/n_-m.gif'},
 {'Date': '2 dec. 2023',
  'Lieu': 'PARIS LA DEFENSE ARENA',
  'Ville': 'NANTERRE',
  'Type': 'CONCERT - POP ROCK FOLK',
  'Prix': '39.00',
  'Artiste': ['STROMAE'],
  'img': 'https://static.ticketmaster.fr/static/images/vignettes/n_stromae.gif'},
 {'Date': '18 au 19 juin 2023',
  'Lieu': 'ZENITH PARIS',
  'Ville': 'PARIS',
  'Type': 'CONCERT - POP ROCK FOLK',
  'Prix': '60.50',
  'Artiste': ['THE BLACK KEYS', 'SPOON'],
  'img': 'https://static.ticketmaster.fr/static/images/vignettes/a_54

In [5]:
df = pd.DataFrame(events)
df

,Date,Lieu,Ville,Type,Prix,Artiste,img
0,28 avr. 2023,ACCOR ARENA,PARIS,CONCERT - VARIETE INTERNATIONALE,39.00,[IBIZA EXPERIENCE],https://static.ticketmaster.fr/static/images/v...
1,1 au 3 juin 2023,ACCOR ARENA,PARIS,CONCERT - VARIETE ET CHANSON FRANCAISE,35.00,[-M-],https://static.ticketmaster.fr/static/images/v...
2,2 dec. 2023,PARIS LA DEFENSE ARENA,NANTERRE,CONCERT - POP ROCK FOLK,39.00,[STROMAE],https://static.ticketmaster.fr/static/images/v...
3,18 au 19 juin 2023,ZENITH PARIS,PARIS,CONCERT - POP ROCK FOLK,60.50,"[THE BLACK KEYS, SPOON]",https://static.ticketmaster.fr/static/images/v...
4,24 au 25 juin 2023,STADE PIERRE MAUROY,VILLENEUVE D ASCQ,CONCERT - RAP HIP-HOP SLAM,40.00,[SOPRANO],https://static.ticketmaster.fr/static/images/v...
5,13 oct. 2023,STADE PIERRE MAUROY,VILLENEUVE D ASCQ,CONCERT - POP ROCK FOLK,49.00,[STROMAE],https://static.ticketmaster.fr/static/images/v...
6,20 mai 2023,PARIS LA DEFENSE ARENA,NANTERRE,CONCERT - VARIETE INTERNATIONALE,63.00,[BURNA BOY],https://static.ticketmaster.fr/static/images/v...
7,28 mars au 5 dec. 2023,ACCOR ARENA,PARIS,CONCERT - RAP HIP-HOP SLAM,40.00,[LOMEPAL],https://static.ticketmaster.fr/static/images/v...
8,11 juil. 2023,GROUPAMA STADIUM,DECINES CHARPIEU,CONCERT - POP ROCK FOLK,79.00,[RED HOT CHILI PEPPERS],https://static.ticketmaster.fr/static/images/v...
9,6 juin 2023,ACCOR ARENA,PARIS,CONCERT - POP ROCK FOLK,49.90,[BRING ME THE HORIZON],https://static.ticketmaster.fr/static/images/v...
